In [11]:
import tools.helpers as th
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

X_train = pd.read_csv('data/bakeoff/Xtrain.csv')
y_train = pd.read_csv('data/bakeoff/ytrain.csv')
X_test = pd.read_csv('data/bakeoff/Xtest.csv')

In [3]:
print(X_train.shape)

(16197, 19)


In [4]:
print(y_train.shape)

(16197, 1)


In [5]:
print(X_test.shape)

(5400, 19)


In [7]:
X_test.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2/20/2015,3,0.75,850,8573,1.0,0.0,0.0,3,6,600,250.0,1945,0.0,98146,47.5030,-122.356,850,8382
1,10/8/2014,3,1.00,1510,6083,1.0,0.0,0.0,4,6,860,650.0,1940,0.0,98115,47.6966,-122.324,1510,5712
2,3/25/2015,4,2.25,1790,42000,1.0,0.0,0.0,3,7,1170,620.0,1983,0.0,98045,47.4819,-121.744,2060,50094
3,2/17/2015,2,1.50,1140,2500,1.0,0.0,1.0,3,7,630,510.0,1988,NaN,98106,47.5707,-122.359,1500,5000
4,5/23/2014,3,1.00,1500,3920,1.0,0.0,0.0,3,7,1000,500.0,1947,0.0,98107,47.6718,-122.359,1640,4017


In [9]:
y_train.head()

,price
0,529000.0
1,253000.0
2,745000.0
3,545000.0
4,390000.0


In [14]:
# log transform continuous variables
to_log = ['price', 'sqft_living', 'dist_from_center', 'lat']
th.log_transform(y_train, 'price')

In [15]:
X_train.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,3/4/2015,3,2.50,1880,4499,2.0,0.0,0.0,3,8,1880,0.0,1993,0.0,98029,47.5664,-121.999,2130,5114
1,10/7/2014,3,2.50,2020,6564,1.0,0.0,0.0,3,7,1310,710.0,1994,0.0,98042,47.3545,-122.158,1710,5151
2,1/16/2015,5,4.00,4720,493534,2.0,0.0,0.0,5,9,3960,760.0,1975,0.0,98027,47.4536,-122.009,2160,219542
3,3/30/2015,2,2.00,1430,3880,1.0,0.0,0.0,4,7,1430,0.0,1949,0.0,98117,47.6844,-122.392,1430,3880
4,10/14/2014,3,2.25,2270,32112,1.0,0.0,0.0,4,8,1740,530.0,1980,0.0,98042,47.3451,-122.094,2310,41606


In [ ]:
# log transform continuous variables
to_log = ['lat', 'dist_from_center', 'sqft_living']
th.log_transform(df_test, to_log)

In [ ]:
y_test_fake = np.full((5400,1), 0)


# fake predictions using the mean of y_train.
your_y_hat_predictions = np.full((5400,1), np.mean(y_train))

r2_score(your_y_hat_predictions, y_test_fake)


In [ ]:
np.savetxt('your_team_member_names.csv', your_y_hat_predictions, delimiter=',')